In [1]:
import json
import pandas as pd
from pandas import json_normalize
from pprint import pprint

### Note:  This file only contains restaurant information for Foursquare and Yelp. I have made separate files for park and pet store.

# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [2]:
import sqlite3 
conn = sqlite3.connect('test.sqlite')

In [3]:

with open('restaurants_fs.json') as f:
    fs = json.load(f)
pprint (fs)


{'context': {'geo_bounds': {'circle': {'center': {'latitude': 39.703014,
                                                  'longitude': -105.078792},
                                       'radius': 1000}}},
 'results': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
                                       'suffix': '.png'},
                              'id': 13145,
                              'name': 'Fast Food Restaurant'}],
              'chains': [{'id': 'f2ee844c-8417-4c9f-80ff-e79f1c44cad4',
                          'name': 'Firehouse Subs'}],
              'distance': 241,
              'fsq_id': '4dc7303b52b1e8f9f7ffb2c7',
              'geocodes': {'main': {'latitude': 39.704777,
                                    'longitude': -105.08075},
                           'roof': {'latitude': 39.704777,
                                    'longitude': -105.08075}},
              'link': '/v3/places/4dc7303b52b1e8f9f7ffb2c7',
        

Parse through the response to get the POI details you want (rating, name, location, etc)

In [4]:
from pandas import json_normalize
json_normalize(fs)
results = json_normalize(fs, record_path='results')
categories = json_normalize(fs['results'], record_path='categories', errors='ignore')

results.rename(columns = {'name':'restaurant_name', 'location.formatted_address':'location_address'}, 
               inplace = True)
categories = json_normalize(fs['results'], record_path='categories', 
                            meta=['restaurant_name','location_address'], errors='ignore')

fs_res_results = results[['restaurant_name','location_address', 'distance']]
fs_res_results.head()

,restaurant_name,location_address,distance
0,Firehouse Subs,"650 S Wadsworth Blvd, Lakewood, CO 80226",241
1,Red Lobster,"810 S Wadsworth Blvd, Lakewood, CO 80226",177
2,Panera Bread,"650 S Wadsworth Blvd, Lakewood, CO 80226",282
3,Chick-Fil-A,"565 S Vance St, Lakewood, CO 80226",428
4,Chuy's,"499 S Vance St, Lakewood, CO 80226",488


In [5]:
categorylist = []
for i in range(len(results['restaurant_name'])):
    categorylist.append(fs['results'][i]['categories'][0]['name'])

cat_list = pd.DataFrame(categorylist)
cat_list.columns = ['category']

Put your parsed results into a DataFrame

In [6]:
restaurant = pd.concat([fs_res_results, cat_list], axis=1, join='inner')
restaurant.sort_values(by = 'distance')


,restaurant_name,location_address,distance,category
1,Red Lobster,"810 S Wadsworth Blvd, Lakewood, CO 80226",177,Seafood Restaurant
0,Firehouse Subs,"650 S Wadsworth Blvd, Lakewood, CO 80226",241,Fast Food Restaurant
2,Panera Bread,"650 S Wadsworth Blvd, Lakewood, CO 80226",282,Fast Food Restaurant
3,Chick-Fil-A,"565 S Vance St, Lakewood, CO 80226",428,Fast Food Restaurant
10,Panda Express,"7660 W Virginia Ave (at Wadsworth Blvd.), Lake...",468,Fast Food Restaurant
4,Chuy's,"499 S Vance St, Lakewood, CO 80226",488,Bar
6,Carl's Jr.,"497 S Wadsworth Blvd, Lakewood, CO 80226",559,Fast Food Restaurant
7,Johnny's New York Pizza & Pasta,"1000 S Wadsworth Blvd, Lakewood, CO 80226",592,Pizzeria
21,Menya Noodle Bar,"450 S Teller St, Lakewood, CO 80226",595,Ramen Restaurant
5,Tstreet Roadhouse,"445 S Teller St, Lakewood, CO 80226",603,Bar


In [7]:
restaurant['restaurant_name'] = restaurant['restaurant_name'].astype('str')

In [8]:
restaurant['location_address'] = restaurant['location_address'].astype('str')

In [9]:
restaurant.to_sql(name='restaurant', con=conn, if_exists='replace', index=False)

In [10]:
pd.read_sql('select * from restaurant', conn)

,restaurant_name,location_address,distance,category
0,Firehouse Subs,"650 S Wadsworth Blvd, Lakewood, CO 80226",241,Fast Food Restaurant
1,Red Lobster,"810 S Wadsworth Blvd, Lakewood, CO 80226",177,Seafood Restaurant
2,Panera Bread,"650 S Wadsworth Blvd, Lakewood, CO 80226",282,Fast Food Restaurant
3,Chick-Fil-A,"565 S Vance St, Lakewood, CO 80226",428,Fast Food Restaurant
4,Chuy's,"499 S Vance St, Lakewood, CO 80226",488,Bar
5,Tstreet Roadhouse,"445 S Teller St, Lakewood, CO 80226",603,Bar
6,Carl's Jr.,"497 S Wadsworth Blvd, Lakewood, CO 80226",559,Fast Food Restaurant
7,Johnny's New York Pizza & Pasta,"1000 S Wadsworth Blvd, Lakewood, CO 80226",592,Pizzeria
8,The French Press,"7323 W Alaska Dr, Lakewood, CO 80226",682,CafÃ©
9,STACK Subs,"420 S Teller St (Alaska Drive), Lakewood, CO 8...",662,Sandwich Restaurant


# Yelp

Send a request with the same location paramaters (location, radius, etc)

In [11]:
with open('restaurants_yelp.json') as y:
    yelp = json.load(y)
pprint (yelp)

{'businesses': [{'alias': 'tstreet-lakewood',
                 'categories': [{'alias': 'bars', 'title': 'Bars'},
                                {'alias': 'desserts', 'title': 'Desserts'},
                                {'alias': 'breakfast_brunch',
                                 'title': 'Breakfast & Brunch'}],
                 'coordinates': {'latitude': 39.70826430562294,
                                 'longitude': -105.07701244110454},
                 'display_phone': '(303) 936-5179',
                 'distance': 603.328790817187,
                 'id': 'bzDDEe41whQBUCxO8012nQ',
                 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/ClTbBjJ8dh3AZN5W8YNVEg/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '445 S Teller St',
                              'address2': '',
                              'address3': '',
                              'city': 'Lakewood',
                              'country': 'US',
             

Parse through your result and get POI details

In [12]:
from pandas import json_normalize
json_normalize(yelp)
results = json_normalize(yelp, record_path='businesses')
results.rename(columns = {'name':'restaurant_name', 'location.display_address':'location_address'}, 
               inplace = True)
yelp_res = results[['restaurant_name','location_address','distance','rating','review_count']]
yelp_res

,restaurant_name,location_address,distance,rating,review_count
0,Tstreet,"[445 S Teller St, Lakewood, CO 80226]",603.328791,4.0,532
1,G-Que BBQ - Lakewood,"[7085 W Alaska Dr, Lakewood, CO 80226]",780.967404,5.0,28
2,Muggle Pub,"[7260 W Alaska Dr, Lakewood, CO 80226]",650.388908,4.5,2
3,Menya Noodle Bar,"[450 S Teller St, Denver, CO 80226]",599.559729,4.0,56
4,Poke Me,"[7357 W Alaska Dr, Lakewood, CO 80226]",708.990763,5.0,19
5,Chuy's,"[499 S Vance St, Lakewood, CO 80226]",500.860107,3.5,255
6,Birdcall,"[444 S Wadsworth Blvd, Lakewood, CO 80226]",649.506911,1.0,1
7,Bar Louie - Belmar,"[7111 W Alaska Dr, Lakewood, CO 80226]",776.070005,3.0,298
8,The Rock Wood Fired Pizza,"[7399 W Alaska Dr, Denver, CO 80226]",714.596732,3.5,482
9,STACK Subs,"[420 S Teller St, Lakewood, CO 80226]",665.416985,4.5,181


In [13]:
categorylist = []
for i in range(len(results['restaurant_name'])):
    categorylist.append(yelp['businesses'][i]['categories'][0]['title'])

cat_list = pd.DataFrame(categorylist)
cat_list.columns = ['category']

Put your parsed results into a DataFrame

In [14]:
restaurant = pd.concat([yelp_res, cat_list], axis=1, join='inner')
restaurant['distance'] = round(restaurant['distance'])
restaurant.sort_values(by = 'distance').head(10)


,restaurant_name,location_address,distance,rating,review_count,category
23,Red Lobster,"[810 S Wadsworth Blvd, Next To Bell Mar Mall, ...",185.0,3.0,99,Seafood
25,Firehouse Subs,"[650 S Wadsworth Blvd, Ste 1, Lakewood, CO 80226]",258.0,3.5,69,Sandwiches
33,Panera Bread,"[650 S. Wadsworth Blvd., Lakewood, CO 80226]",283.0,2.0,115,Sandwiches
28,Chick-fil-A,"[565 S Vance St, Lakewood, CO 80226]",429.0,3.0,77,Fast Food
35,Panda Express,"[7660 W. Virginia Ave Unit D, Lakewood, CO 80226]",461.0,2.5,30,Fast Food
5,Chuy's,"[499 S Vance St, Lakewood, CO 80226]",501.0,3.5,255,Tex-Mex
41,Carl's Jr,"[497 S Wadsworth, Lakewood, CO 80226]",560.0,2.5,48,Fast Food
14,Johnny's New York Pizza & Pasta,"[1000 S Wadsworth Blvd, Lakewood, CO 80226]",590.0,4.0,238,Pizza
3,Menya Noodle Bar,"[450 S Teller St, Denver, CO 80226]",600.0,4.0,56,Ramen
0,Tstreet,"[445 S Teller St, Lakewood, CO 80226]",603.0,4.0,532,Bars


In [16]:
restaurant['restaurant_name'] = restaurant['restaurant_name'].astype('str')
restaurant['location_address'] = restaurant['location_address'].astype('str')

In [17]:
restaurant.to_sql(name='restaurant_yelp', con=conn, if_exists='replace', index=False)

In [18]:
pd.read_sql('select * from restaurant_yelp', conn)

,restaurant_name,location_address,distance,rating,review_count,category
0,Tstreet,"['445 S Teller St', 'Lakewood, CO 80226']",603.0,4.0,532,Bars
1,G-Que BBQ - Lakewood,"['7085 W Alaska Dr', 'Lakewood, CO 80226']",781.0,5.0,28,Southern
2,Muggle Pub,"['7260 W Alaska Dr', 'Lakewood, CO 80226']",650.0,4.5,2,Themed Cafes
3,Menya Noodle Bar,"['450 S Teller St', 'Denver, CO 80226']",600.0,4.0,56,Ramen
4,Poke Me,"['7357 W Alaska Dr', 'Lakewood, CO 80226']",709.0,5.0,19,Poke
5,Chuy's,"['499 S Vance St', 'Lakewood, CO 80226']",501.0,3.5,255,Tex-Mex
6,Birdcall,"['444 S Wadsworth Blvd', 'Lakewood, CO 80226']",650.0,1.0,1,Chicken Shop
7,Bar Louie - Belmar,"['7111 W Alaska Dr', 'Lakewood, CO 80226']",776.0,3.0,298,Bars
8,The Rock Wood Fired Pizza,"['7399 W Alaska Dr', 'Denver, CO 80226']",715.0,3.5,482,Pizza
9,STACK Subs,"['420 S Teller St', 'Lakewood, CO 80226']",665.0,4.5,181,Sandwiches


# Google (stretch)

Use the same process as the first two APIs

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Get the top 10 restaurants according to their rating

# Travelling Salesman Problem (stretch)

If you have time, follow the steps in the [ortools tutorial](https://developers.google.com/optimization/routing/tsp) using Google's [Directions API](https://developers.google.com/maps/documentation/directions/start).